# CMSC320 Final Project
## Kinsey Smith, Sarah Bullard, Yiwen Shen

In [33]:
from IPython.display import HTML, display
display(HTML("<table><tr><td><img src='blm_twitter_image.jpg'></td><td><img src='Twitter_bird_logo_2012.svg.png'></td><td><img src='trump_twitter_image.jpg'></td></tr></table>"))

,,


<i> how to make these images less ugly? <i/>

### Introduction
Our project is surrounding the Twitter accounts of the Black Lives Matter Movement (@Blklivesmatter) and Donald Trump (@realDonaldTrump). We focused on the sentiment of the tweets of each account versus each individual tweet's replies. Sentiment analysis is a way of classifying a text as positive, negative, or neutral using text analysis. <i>FIX THAT SENTENCE</i>. However, because of the complex and sarcastic components of the English language, sentiment analysis is not a sole way of categorizing something as positive or negative. Because of this, we needed other factors to tell the sentiment of a reply. In order to do this, we created a feature vector and used that to classify the sentiment of the reply and used SVM machine learning in order to have the machine classify it for us. <i>FIX THAT SENTENCE</i>

The features in our feature vector are as follows. 

Our first feature was the original sentiment analysis, because although it is not reliably conclusive on its own, it can tell us something about the mood of the sentence. 

The second feature we worked on focused on the user who posted the reply to the specific tweet. We checked whether or not the user was following other accounts that aligned with Donald Trump's or BLM's views, including politicians of either party.

<b> Fix the two sentences above and continue adding features when we have them. Include issues we ran into and problems we faced/solutions we came up with as well? </b>


In [2]:
import tweepy
import textblob
import numpy as np
import pandas as pd
import collections
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

/Users/kinseyanna/anaconda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### Accessing Twitter's Data
In order to access Twitter's API, we had to create applications and personally get authentication tokens. Even though we are all allowed access to Twitter, we cannot give out these confidential tokens on this public notebook. In order to get past this hurdle, we created a function that would pull from our own files on our own machines for these tokens. In the cell below is a copy of credentials.py, without the confidential information. 

In [3]:
from credentials import *

In [4]:
#A function that takes these credentials and sets up the API.
def api_setup():
    authentication = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    authentication.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tweepy.API(authentication)
    return api

In [5]:
# Extracting the tweets
extract_tweets = api_setup()

### Collecting the Data in a Functional Way

Now we must start by getting the replies of each account, linked to the tweet they are replying to. We use the tweepy and collections libraries to do this. 

In [6]:
# Donald Trump Replies
tweet_ids_donald = []
for page in tweepy.Cursor(extract_tweets.user_timeline,screen_name="realDonaldTrump").pages(20):
    for item in page:
        tweet_ids_donald.append(item.id_str)


<b> To-do: figure out how to get tweets between a certain timeline (2 years earlier max) from this method ^ </b>

In [7]:
replies_donald = collections.defaultdict(list)

In [31]:
query = tweepy.Cursor(extract_tweets.search,q="to:realDonaldTrump").items(5000)

<b> To-do: Change the number of items pulled???  ^^^ </b>

In [32]:
for tweet in query:
    if replies_donald.get(tweet.in_reply_to_status_id_str) != None:
        key = tweet.in_reply_to_status_id_str
        replies_donald[key] = replies_donald[key].append(tweet)

TweepError: Twitter error response: status code = 429

<b><i> I figured out how to work with a dictionary when putting it into a dataframe and doing sentiment analysis on that, but for some reason I keep getting this error, even after waiting 15 minutes. I'll try running it again tomorrow, but I think it has something to do with the fact that we're pulling from so many tweets. <b/><i/>

In [30]:
print(len(replies_donald.keys()))

0


In [11]:
data_donald = pd.DataFrame(data=[tweet.text for tweet in replies_donald.values()], columns=["text"])
data_donald["length"] = [len(tweet.text) for tweet in replies_donald.values()]
data_donald["in_reply_to"] = [id_t for id_t,tweet in replies_donald]
data_donald["ID"] = [tweet.id for tweet in replies_donald.values()]
data_donald["date created"] = [tweet.created_at for tweet in replies_donald.values()]

In [12]:
# Black Lives Matter's Tweets and their Replies
replies_blm = collections.defaultdict(list)

In [13]:
query2 = tweepy.Cursor(extract_tweets.search,q="to:Blklivesmatter").items(20000)

In [21]:
for tweet in query2:
    if replies_blm.get(tweet.in_reply_to_status_id_str) != None:
        key = tweet.in_reply_to_status_id_str
        replies_blm[key] = replies_blm[key].append(tweet)

In [22]:
data_blm = pd.DataFrame(data=[tweet.text for tweet in replies_blm.values()], columns=["text"])
data_blm["length"] = [len(tweet.text) for tweet in replies_blm.values()]
data_blm["in_reply_to"] = [id_t for id_t,tweet in replies_blm]
data_blm["ID"] = [tweet.id for tweet in replies_blm.values()]
data_blm["date created"] = [tweet.created_at for tweet in replies_blm.values()]

In order to get around Twitter's rule of no more than 15 queries in 15 minutes and to efficiently pull from a specific and unchanging dataset, we put the data of each account and their replies into a csv starting from (<b> DATE TIME OF DATA PULLED INTO CSV</b>). Because we only needed certain data from each tweet, we only added (<b> COLUMNS THAT WE PULLED FROM DATAFRAME INTO CSV</b>).

In [23]:
# Work on getting it into csv or json file here!

#### Sentiment Analysis

As explained before, sentiment analysis is a basic way of analyzing a text to tell if it has a positive, negative, or neutral sentiment. NLTK is the library we will use to do this section. <b> TODO: add more to this section </b>

## 468 words at this point. 

In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [25]:
sia = SentimentIntensityAnalyzer()

In [26]:
sentiment_nltk_donald = []
for _,x in data_donald.iterrows():
    sentiment_nltk.append((x["text"],sia.polarity_scores(x['text'])))
sentiment_nltk_blm = []
for _,x in data_blm.iterrows():
    sentiment_nltk.append((x["text"],sia.polarity_scores(x['text'])))

In [27]:
i = 0
while i < 11:
    print(sentiment_nltk_donald[i])
    i = i + 1

IndexError: list index out of range